In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark


In [7]:
textFile_1 = spark.read.text('sample_libsvm_data.txt')

In [8]:
textFile_1

DataFrame[value: string]

In [5]:
# import pandas as pd
# textFile = pd.read_table('test.txt')
# print(textFile)

In [10]:
textFile_2 = spark.read.text('test.txt')

In [11]:
print(textFile_2)

DataFrame[value: string]


In [52]:
textFile.count()  # num of rows in this DataFrame

4

In [53]:
textFile.first() # first row in this DataFrame

Row(value='hello world')

In [11]:
line_with_world = textFile.filter(textFile.value.contains('world'))

In [12]:
print(line_with_world)

DataFrame[value: string]


In [13]:
line_with_world.first()

Row(value='hello world')

In [14]:
textFile.filter(textFile.value.contains('hello')).count()

4

# [Example_1: Estimator, Transformer, and Param]('http://spark.apache.org/docs/latest/ml-pipeline.html')



In [22]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pprint import pprint

In [23]:
# prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LR instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)

# print out the parameters, documentations, and ant default values.
pprint('LR param:\n' + lr.explainParams() + '\n')


('LR param:\n'
 'aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)\n'
 'elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha '
 '= 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. '
 '(default: 0.0)\n'
 'family: The name of family which is a description of the label distribution '
 'to be used in the model. Supported options: auto, binomial, multinomial '
 '(default: auto)\n'
 'featuresCol: features column name. (default: features)\n'
 'fitIntercept: whether to fit an intercept term. (default: True)\n'
 'labelCol: label column name. (default: label)\n'
 'lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under '
 'bound constrained optimization. The bound matrix must be compatible with the '
 'shape (1, number of features) for binomial regression, or (number of '
 'classes, number of features) for multinomial regression. (undefined)\n'
 'lowerBoundsOnIntercepts: The lower bounds on intercepts if fitt

In [19]:
# Learn a LR model. This uses the parameters stored in lr.
model1 = lr.fit(training)

In [24]:
print('Model 1 was fit using params: ')
pprint(model1.extractParamMap())

Model 1 was fit using params: 
{Param(parent='LogisticRegression_42649ca88ea1827dd408', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_42649ca88ea1827dd408', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0,
 Param(parent='LogisticRegression_42649ca88ea1827dd408', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_42649ca88ea1827dd408', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_42649ca88ea1827dd408', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LogisticRegression_42649ca88ea1827dd408', name='labelCol', doc='label column name'): 'label',
 Param(parent='LogisticRegress

In [25]:
# we may alternatively specify parameteres using a Python dictionary as a paramMap
# 3 different methods

paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # specify 1 param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # specify multiple params

In [26]:
# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
pprint(model2.extractParamMap())


Model 2 was fit using parameters: 
{Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0,
 Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto',
 Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='labelCol', doc='label column name'): 'label',
 Param(parent='LogisticRegression_4fa5bd460623221dc8d1', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LogisticReg

In [27]:
# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [32]:
# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.

prediction = model2.transform(test)
pprint(prediction)
prediction.first()


DataFrame[label: double, features: vector, rawPrediction: vector, myProbability: vector, prediction: double]


Row(label=1.0, features=DenseVector([-1.0, 1.5, 1.3]), rawPrediction=DenseVector([-2.8047, 2.8047]), myProbability=DenseVector([0.0571, 0.9429]), prediction=1.0)

In [33]:
result = prediction.select("features", "label", "myProbability", "prediction").collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.057073041710340174,0.9429269582896599], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9238522311704104,0.07614776882958973], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.10972776114779419,0.8902722388522057], prediction=1.0


# [Example_2: Pipeline]('http://spark.apache.org/docs/latest/ml-pipeline.html#example-pipeline')

In [44]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer


In [45]:
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [46]:
# configure an ML pipeline, which consists of three stages: tokenizer, hashingTF and lr
tokenizer = Tokenizer(inputCol='text', outputCol='words')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [47]:
# Fit the pipeline to training documents
model = pipeline.fit(training)

In [48]:
# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [49]:
# predict
prediction = model.transform(test)
selected = prediction.select('id', 'text', 'probability', 'prediction')
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.1596407738787475,0.8403592261212525], prediction=1.000000
(5, l m n) --> prob=[0.8378325685476744,0.16216743145232562], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976037,0.9307336686702395], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444218,0.01784246665557808], prediction=0.000000
